### ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical

# 데이터 로드 및 전처리
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# 모델 구축
model = Sequential([
    Flatten(input_shape=(28, 28)),  # 이미지를 1차원으로 펼치기
    Dense(128, activation='relu'),  # 첫 번째 은닉층
    Dense(64, activation='relu'),   # 두 번째 은닉층
    Dense(10, activation='softmax') # 출력층 (10개의 클래스)
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Accuracy:", test_accuracy)


### RNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 예제로 사용할 간단한 시계열 데이터 생성
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

# 데이터셋 생성
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

# 모델 구축
model = Sequential([
    SimpleRNN(20, input_shape=[None, 1], return_sequences=True),
    SimpleRNN(20),
    Dense(1)
])

# 모델 컴파일 및 학습
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

# 모델 평가
mse_test = model.evaluate(X_test, y_test)

print("Test MSE:", mse_test)


#### Deep Recurrent Neural Network
#### Bidirectional Recurrent Neural Network

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

# 예제로 사용할 간단한 시계열 데이터 생성
def generate_time_series(batch_size, n_steps):
    freq = np.random.rand(batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - 0.1) * (freq * 10 + 10))  # wave
    series += 0.2 * np.sin((time - 0.3) * (freq * 20 + 20))  # + wave
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)  # + noise
    return series[..., np.newaxis].astype(np.float32)

# 데이터셋 생성
n_steps = 50
series = generate_time_series(1000, n_steps + 1)
X_train, y_train = series[:700, :n_steps], series[:700, -1]
X_valid, y_valid = series[700:900, :n_steps], series[700:900, -1]
X_test, y_test = series[900:, :n_steps], series[900:, -1]

# Bi-RNN 모델 구축
model = Sequential([
    Bidirectional(LSTM(20, return_sequences=True), input_shape=[None, 1]),
    Bidirectional(LSTM(20)),
    Dense(1)
])

# 모델 컴파일 및 학습
model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

# 테스트 세트에 대한 예측 수행
y_pred = model.predict(X_test)

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('Bi-RNN Predictions')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.show()


#### CNN MNIST

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# 데이터 불러오기 및 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 모델 구축
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

# 모델 평가
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/10

469/469 [==============================] - 11s 22ms/step - loss: 0.2110 - accuracy: 0.9364 - val_loss: 0.0593 - val_accuracy: 0.9826
Epoch 2/10
469/469 [==============================] - 11s 24ms/step - loss: 0.0568 - accuracy: 0.9827 - val_loss: 0.0507 - val_accuracy: 0.9830
Epoch 3/10
469/469 [==============================] - 12s 25ms/step - loss: 0.0387 - accuracy: 0.9880 - val_loss: 0.0364 - val_accuracy: 0.9861
Epoch 4/10
469/469 [==============================] - 13s 29ms/step - loss: 0.0301 - accuracy: 0.9905 - val_loss: 0.0337 - val_accuracy: 0.9900
Epoch 5/10
469/469 [==============================] - 13s 28ms/step - loss: 0.0240 - accuracy: 0.9922 - val_loss: 0.0411 - val_accuracy: 0.9855
Epoch 6/10
469/469 [==============================] - 14s 29ms/step - loss: 0.0199 - accuracy: 0.9936 - val_loss: 0.0285 - val_accuracy: 0.9903
Epoch 7/10
469/469 [==============================] - 14s 30ms/step - loss: 0.0151 - accuracy: 0.9953 - val_loss: 0.0269 - val_accurac